In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [1]:
!git clone https://github.com/totalgood/nlpia.git

Cloning into 'nlpia'...
remote: Enumerating objects: 196, done.
remote: Counting objects: 100% (196/196), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 5827 (delta 98), reused 138 (delta 57), pack-reused 5631
Receiving objects: 100% (5827/5827), 124.02 MiB | 26.22 MiB/s, done.
Resolving deltas: 100% (3674/3674), done.


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
df = pd.read_csv('/content/nlpia/src/nlpia/data/moviedialog.csv')

In [0]:
df.drop(columns='Unnamed: 0', axis=1,inplace=True)

In [5]:
df.head(5)

,statement,reply
0,you're asking me out. that's so cute. what's y...,forget it.
1,"no, no, it's my fault we didn't have a proper ...",cameron.
2,"gosh, if only we could find kat a boyfriend...",let me see what i can do.
3,c'esc ma tete. this is my head,right. see? you're ready for the quiz.
4,how is our little find the wench a date plan p...,"well, there's someone i think might be"


In [0]:
input_texts, target_texts = [], []
input_vocabulary = set()
output_vocabulary = set()
start_token = '\t'
stop_token = '\n'
max_training_samples = min(25000, len(df) -1)

In [0]:
for input_text, target_text in zip(df.statement, df.reply):
  target_text = start_token + target_text + stop_token
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    input_vocabulary.add(char)
  for char in target_text:
    output_vocabulary.add(char) 

In [0]:
input_vocabulary = sorted(input_vocabulary)
output_vocabulary = sorted(output_vocabulary)

In [0]:
input_vocab_size = len(input_vocabulary)
output_vocab_size = len(output_vocabulary)

In [0]:
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [0]:
input_token_index = dict((c,i) for i,c in enumerate(input_vocabulary))
target_token_index = dict((c,i) for i,c in enumerate(output_vocabulary))

reverse_input_char_index = dict((i, c) for c, i in input_token_index.items())
reverse_target_char_index = dict((i, c) for c, i in target_token_index.items())

In [0]:
import numpy as np
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, input_vocab_size), dtype = 'float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, output_vocab_size), dtype = 'float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, output_vocab_size), dtype='float32')

In [0]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t,c in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[c]] = 1.
  for t,c in enumerate(target_text):
    decoder_input_data[i, t, target_token_index[c]] = 1.
    if t > 0:
      decoder_target_data[i, t-1, target_token_index[c]] = 1.

In [14]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

batch_size = 64
epochs = 100
num_neurons = 256

encoder_inputs = Input(shape=(None, input_vocab_size))
encoder = LSTM(num_neurons, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, output_vocab_size))
decoder_lstm = LSTM(num_neurons, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(output_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split=0.1)

Using TensorFlow backend.





Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 44)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 46)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 308224      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  310272      input_2[0][0]                    
                                                                 lstm_1[0][1]            

In [0]:
# Assemble the model for sequence generation
encoder_model = Model(encoder_inputs, encoder_states)
thought_input = [Input(shape=(num_neurons,)), Input(shape=(num_neurons,))]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=thought_input)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(inputs=[decoder_inputs] + thought_input,
                     outputs=[decoder_outputs] + decoder_states)

In [0]:
# Build a character-based translator 
def decode_sequence(input_seq):
  thought = encoder_model.predict(input_seq)
  
  target_seq = np.zeros((1,1,output_vocab_size))
  target_seq[0, 0, target_token_index[start_token]] = 1.
  stop_condition = False
  generated_sequence = ''
  
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + thought)
    generated_token_idx = np.argmax(output_tokens[0, -1, :])
    generated_char = reverse_target_char_index[generated_token_idx]
    generated_sequence += generated_char
    if (generated_char == stop_token or len(generated_sequence) > max_decoder_seq_length):
      stop_condition = True
    target_seq = np.zeros((1, 1, output_vocab_size))
    target_seq[0, 0, generated_token_idx] = 1.
    thought = [h, c]
  return generated_sequence

In [0]:
# Generating a response
def response(input_text):
  input_seq = np.zeros((1, max_encoder_seq_length, input_vocab_size), dtype='float32')
  for t, char in enumerate(input_text):
    input_seq[0, t, input_token_index[char]] = 1.
  decoded_sentence = decode_sequence(input_seq)
  print('Bot Reply (Decoded sentence):', decoded_sentence)